## Generating All Possible Derivative Assignments for a Given Multiset of Fields and Derivatives

In [1]:
# from http://code.activestate.com/recipes/578608-generates-tuples-of-integers-with-a-given-sum/
from itertools import permutations

def tuples_sum(nbval,total,order=True) :
    """ 
        Generate all the tuples L of nbval positive or nul integer 
        such that sum(L)=total.
        The tuples may be ordered (decreasing order) or not
    """
    if nbval == 0 and total == 0 : yield tuple() ; return #raise StopIteration
    if nbval == 1 : yield (total,) ; return #raise StopIteration
    if total==0 : yield (0,)*nbval ; return #raise StopIteration
    for start in range(total,0,-1) :
        for qu in tuples_sum(nbval-1,total-start) :
            if qu[0]<=start :
                sol=(start,)+qu
                if order : yield sol
                else :
                    l=set()
                    for p in permutations(sol,len(sol)) :
                        if p not in l :
                            l.add(p)
                            yield p
    
if __name__=='__main__' :
    print("How to obtain 5 by adding a+b+c (>=0) ? ")
    print("Give me the list of (a,b,c) tuples.")
    g=tuples_sum(3,6,order=False)
    print(list(g))
    
    print("How to obtain 6 by adding 3 positive or nul integers ?")
    g=tuples_sum(3,6,order=True)
    print(list(g))

How to obtain 5 by adding a+b+c (>=0) ? 
Give me the list of (a,b,c) tuples.
[(6, 0, 0), (0, 6, 0), (0, 0, 6), (5, 1, 0), (5, 0, 1), (1, 5, 0), (1, 0, 5), (0, 5, 1), (0, 1, 5), (4, 2, 0), (4, 0, 2), (2, 4, 0), (2, 0, 4), (0, 4, 2), (0, 2, 4), (4, 1, 1), (1, 4, 1), (1, 1, 4), (3, 3, 0), (3, 0, 3), (0, 3, 3), (3, 2, 1), (3, 1, 2), (2, 3, 1), (2, 1, 3), (1, 3, 2), (1, 2, 3), (2, 2, 2)]
How to obtain 6 by adding 3 positive or nul integers ?
[(6, 0, 0), (5, 1, 0), (4, 2, 0), (4, 1, 1), (3, 3, 0), (3, 2, 1), (2, 2, 2)]


In [2]:
n = 4
m = 7
list(tuples_sum(m,n,order=False))


[(4, 0, 0, 0, 0, 0, 0),
 (0, 4, 0, 0, 0, 0, 0),
 (0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 4, 0, 0, 0),
 (0, 0, 0, 0, 4, 0, 0),
 (0, 0, 0, 0, 0, 4, 0),
 (0, 0, 0, 0, 0, 0, 4),
 (3, 1, 0, 0, 0, 0, 0),
 (3, 0, 1, 0, 0, 0, 0),
 (3, 0, 0, 1, 0, 0, 0),
 (3, 0, 0, 0, 1, 0, 0),
 (3, 0, 0, 0, 0, 1, 0),
 (3, 0, 0, 0, 0, 0, 1),
 (1, 3, 0, 0, 0, 0, 0),
 (1, 0, 3, 0, 0, 0, 0),
 (1, 0, 0, 3, 0, 0, 0),
 (1, 0, 0, 0, 3, 0, 0),
 (1, 0, 0, 0, 0, 3, 0),
 (1, 0, 0, 0, 0, 0, 3),
 (0, 3, 1, 0, 0, 0, 0),
 (0, 3, 0, 1, 0, 0, 0),
 (0, 3, 0, 0, 1, 0, 0),
 (0, 3, 0, 0, 0, 1, 0),
 (0, 3, 0, 0, 0, 0, 1),
 (0, 1, 3, 0, 0, 0, 0),
 (0, 1, 0, 3, 0, 0, 0),
 (0, 1, 0, 0, 3, 0, 0),
 (0, 1, 0, 0, 0, 3, 0),
 (0, 1, 0, 0, 0, 0, 3),
 (0, 0, 3, 1, 0, 0, 0),
 (0, 0, 3, 0, 1, 0, 0),
 (0, 0, 3, 0, 0, 1, 0),
 (0, 0, 3, 0, 0, 0, 1),
 (0, 0, 1, 3, 0, 0, 0),
 (0, 0, 1, 0, 3, 0, 0),
 (0, 0, 1, 0, 0, 3, 0),
 (0, 0, 1, 0, 0, 0, 3),
 (0, 0, 0, 3, 1, 0, 0),
 (0, 0, 0, 3, 0, 1, 0),
 (0, 0, 0, 3, 0, 0, 1),
 (0, 0, 0, 1, 3, 0, 0),
 (0, 0, 0, 1, 0,

In [3]:
m = 2
n = 5
list(tuples_sum(m,n,order=False))
   

[(5, 0), (0, 5), (4, 1), (1, 4), (3, 2), (2, 3)]

In [4]:
#import inspect # this is to check recursion depth for debugging

def generate_pair_partitions(inner_partition, i_start):
    # INPUT:
    # inner_partition: sorted list (non-increasing order) of numbers that sum to some n_Di, an element of the outer
    # partition of n_D. 
    # OUTPUT:
    # pair_partitions_list: a list of lists of tuples, where each tuple has two elements that sum to the corresponding
    # element of the inner_partition. The number of tuples in each sublist matches the number of elements in 
    # inner_partition. 
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to partition elements of inner partition into two
    # non-negative integers (indicating number of derivatives acting on psi_bar and psi, respectively). 

    # base case
    #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
    if len(inner_partition) == 1:
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list = [[item] for item in pair_partitions]
        return pair_partitions_list
    
    # if two successive elements of inner_partition are equal, then different orderings of 
    # corresponding pair partitions [..., (a,b), (c,d), ...] and [..., (c,d), (a,b), ...] are equal. To avoid
    # repeats, need to modify recursion step for case when successive elements are equal. 
    if inner_partition[1] == inner_partition[0]:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
        for i in range(i_start, len(pair_partitions)):
            pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], i)
            #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    
    # recursion step is simpler in case where successive elements are not equal
    else:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], 0)
        for i in range(len(pair_partitions)):
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    


In [5]:
inner_partition = [3, 2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (2, 0), (2, 0), (1, 0)],
 [(3, 0), (2, 0), (2, 0), (0, 1)],
 [(3, 0), (2, 0), (0, 2), (1, 0)],
 [(3, 0), (2, 0), (0, 2), (0, 1)],
 [(3, 0), (2, 0), (1, 1), (1, 0)],
 [(3, 0), (2, 0), (1, 1), (0, 1)],
 [(3, 0), (0, 2), (0, 2), (1, 0)],
 [(3, 0), (0, 2), (0, 2), (0, 1)],
 [(3, 0), (0, 2), (1, 1), (1, 0)],
 [(3, 0), (0, 2), (1, 1), (0, 1)],
 [(3, 0), (1, 1), (1, 1), (1, 0)],
 [(3, 0), (1, 1), (1, 1), (0, 1)],
 [(0, 3), (2, 0), (2, 0), (1, 0)],
 [(0, 3), (2, 0), (2, 0), (0, 1)],
 [(0, 3), (2, 0), (0, 2), (1, 0)],
 [(0, 3), (2, 0), (0, 2), (0, 1)],
 [(0, 3), (2, 0), (1, 1), (1, 0)],
 [(0, 3), (2, 0), (1, 1), (0, 1)],
 [(0, 3), (0, 2), (0, 2), (1, 0)],
 [(0, 3), (0, 2), (0, 2), (0, 1)],
 [(0, 3), (0, 2), (1, 1), (1, 0)],
 [(0, 3), (0, 2), (1, 1), (0, 1)],
 [(0, 3), (1, 1), (1, 1), (1, 0)],
 [(0, 3), (1, 1), (1, 1), (0, 1)],
 [(2, 1), (2, 0), (2, 0), (1, 0)],
 [(2, 1), (2, 0), (2, 0), (0, 1)],
 [(2, 1), (2, 0), (0, 2), (1, 0)],
 [(2, 1), (2, 0), (0, 2), (0, 1)],
 [(2, 1), (2, 0), (1

In [6]:
inner_partition = [3, 3, 2, 2]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (3, 0), (2, 0), (2, 0)],
 [(3, 0), (3, 0), (2, 0), (0, 2)],
 [(3, 0), (3, 0), (2, 0), (1, 1)],
 [(3, 0), (3, 0), (0, 2), (0, 2)],
 [(3, 0), (3, 0), (0, 2), (1, 1)],
 [(3, 0), (3, 0), (1, 1), (1, 1)],
 [(3, 0), (0, 3), (2, 0), (2, 0)],
 [(3, 0), (0, 3), (2, 0), (0, 2)],
 [(3, 0), (0, 3), (2, 0), (1, 1)],
 [(3, 0), (0, 3), (0, 2), (0, 2)],
 [(3, 0), (0, 3), (0, 2), (1, 1)],
 [(3, 0), (0, 3), (1, 1), (1, 1)],
 [(3, 0), (2, 1), (2, 0), (2, 0)],
 [(3, 0), (2, 1), (2, 0), (0, 2)],
 [(3, 0), (2, 1), (2, 0), (1, 1)],
 [(3, 0), (2, 1), (0, 2), (0, 2)],
 [(3, 0), (2, 1), (0, 2), (1, 1)],
 [(3, 0), (2, 1), (1, 1), (1, 1)],
 [(3, 0), (1, 2), (2, 0), (2, 0)],
 [(3, 0), (1, 2), (2, 0), (0, 2)],
 [(3, 0), (1, 2), (2, 0), (1, 1)],
 [(3, 0), (1, 2), (0, 2), (0, 2)],
 [(3, 0), (1, 2), (0, 2), (1, 1)],
 [(3, 0), (1, 2), (1, 1), (1, 1)],
 [(0, 3), (0, 3), (2, 0), (2, 0)],
 [(0, 3), (0, 3), (2, 0), (0, 2)],
 [(0, 3), (0, 3), (2, 0), (1, 1)],
 [(0, 3), (0, 3), (0, 2), (0, 2)],
 [(0, 3), (0, 3), (0

In [7]:
inner_partition = [2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (1, 0)],
 [(2, 0), (2, 0), (0, 1)],
 [(2, 0), (0, 2), (1, 0)],
 [(2, 0), (0, 2), (0, 1)],
 [(2, 0), (1, 1), (1, 0)],
 [(2, 0), (1, 1), (0, 1)],
 [(0, 2), (0, 2), (1, 0)],
 [(0, 2), (0, 2), (0, 1)],
 [(0, 2), (1, 1), (1, 0)],
 [(0, 2), (1, 1), (0, 1)],
 [(1, 1), (1, 1), (1, 0)],
 [(1, 1), (1, 1), (0, 1)]]

In [8]:
inner_partition = [2, 2]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0)],
 [(2, 0), (0, 2)],
 [(2, 0), (1, 1)],
 [(0, 2), (0, 2)],
 [(0, 2), (1, 1)],
 [(1, 1), (1, 1)]]

In [9]:
inner_partition = (2, 2, 2)
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (1, 1)],
 [(2, 0), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (1, 1)],
 [(2, 0), (1, 1), (1, 1)],
 [(0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (1, 1)],
 [(0, 2), (1, 1), (1, 1)],
 [(1, 1), (1, 1), (1, 1)]]

In [10]:
def generate_extended_pair_partitions(inner_partitions_list, first_is_F):
    # INPUT:
    # inner_partitions_list: list of sorted tuples (each in non-increasing order) where each tuple has m_i positive
    # integers that sum to n_i, where the n_i sum to n, the total number of derivatives. 
    # OUTPUT:
    # extended_pair_partitions_list: list of lists of m_i-tuples of 2-tuples, each sublist an extended pair 
    # partition, only represents Dirac bilinear fields, not F fields. 
    # EXPLANATION:
    # recursively generates a list of all sequences (lists) of pair partitions, where each pair partition is an m_i-tuple of 
    # 2-tuples, one m_i-tuple for each fielnd type. each sequence contains one pair partition for each
    # of the tuples in inner_partitions_list (one such tuple serves as an argument for generate_pair_partitions()).
    # if first inner partition in inner_partition_list corresponds to distribution of derivatives among F-type fields, omit
    # first partition
    
    # remove any inner partition for F-type fields
    if first_is_F == True:
        inner_partitions_list = inner_partitions_list[1:]
    
    # base case
    if len(inner_partitions_list) == 1:
        partition = inner_partitions_list[0]
        pair_partitions_list = generate_pair_partitions(partition, 0)
        extended_pair_partitions_list = [[pair_partitions_list[i]] for i in range(len(pair_partitions_list))]
        return extended_pair_partitions_list
    
    # main body
    extended_pair_partitions_list = []
    first_partition = inner_partitions_list[0]
    first_pair_partitions_list = generate_pair_partitions(first_partition, 0) # list of all pair partitions of first inner partition
    extended_pair_partitions_list_old = generate_extended_pair_partitions(inner_partitions_list[1:], False)
    for i in range(len(first_pair_partitions_list)):
        extension = first_pair_partitions_list[i]
        for j in range(len(extended_pair_partitions_list_old)):
            extended_pair_partition_old = extended_pair_partitions_list_old[j]
            extended_pair_partition_new = [extension] + extended_pair_partition_old 
            extended_pair_partitions_list.append(extended_pair_partition_new)
    return extended_pair_partitions_list   
        

In [11]:
inner_partitions_list = [[3, 2, 2, 1], [3, 3]]
generate_extended_pair_partitions(inner_partitions_list, False)

[[[(3, 0), (2, 0), (2, 0), (1, 0)], [(3, 0), (3, 0)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(3, 0), (0, 3)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(3, 0), (2, 1)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(3, 0), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(0, 3), (0, 3)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(0, 3), (2, 1)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(0, 3), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(2, 1), (2, 1)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(2, 1), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (1, 0)], [(1, 2), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(3, 0), (3, 0)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(3, 0), (0, 3)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(3, 0), (2, 1)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(3, 0), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(0, 3), (0, 3)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(0, 3), (2, 1)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(0, 3), (1, 2)]],
 [[(3, 0), (2, 0), (2, 0), (0, 1)], [(2, 1), (2, 1)]],
 [[(3, 0),

In [12]:
def generate_inner_partitions(outer_partition, field_multiplicities):
    # INPUT:
    # outer_partition: list of numbers that sum to number of derivatives n_D with n_fields elements. 
    # field_multiplicities: list of multiplicities of fields, same length as outer_partition
    # OUTPUT:
    # subpartitions_list: list of lists of tuples, with each tuple a sorted partition of the corresponding element
    # of outer partition
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to sub-partition elements of outer partition.
    # if there are k_1 distinct ways to partition n1, k_2 ways to partition n2, ..., k_d ways to partition nd,
    # then there are k_1 x k_2 x ... x k_d distinct ways to sub-partition elements of outer partition. 

    if len(outer_partition) != len(field_multiplicities):
        print('Error: Lengths of outer_partition and field_multiplicities arguments must be equal.')
        return None

    # base case
    if len(outer_partition)==1:
        n = outer_partition[0]
        m = field_multiplicities[0]
        return [[subpartition_tuple] for subpartition_tuple in tuples_sum(m, n, order=True)]

    # main body
    subpartitions_list = [] # stores all distinct subpartitions 
    n = outer_partition[0]
    m = field_multiplicities[0]
    inner_partitions = list(tuples_sum(m, n, order=True)) 
    subpartitions = generate_inner_partitions(outer_partition[1:], field_multiplicities[1:])
    for i in range(len(inner_partitions)):
        for j in range(len(subpartitions)):
            subpartition_extended = [inner_partitions[i]] + subpartitions[j] 
            subpartitions_list.append(subpartition_extended)

    return subpartitions_list  

In [13]:
outer_partition = [1, 5, 2]
field_multiplicities = [3, 4, 3]
generate_inner_partitions(outer_partition, field_multiplicities)

[[(1, 0, 0), (5, 0, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (5, 0, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (2, 0, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (1, 1, 0)]]

In [14]:
def generate_derivative_assignments(field_multiset):
    # INPUT:
    # field_multiset: encodes the set of fields and derivatives making up the term
    #     e.g., field_multiset = {D:3, F:2, S:1, T:2};
    #     if a field is absent, simply omit it - e.g., write {D:3, F:2, S:1, T:2}, not {D:3, F:2, S:1, T:2, V:0}
    # OUTPUT:
    # derivative_assignments_list: A dictionary of lists of lists of tuples, where each sublist of each list is a 
    # distinct assignment of derivatives to fields.
    # EXPLANATION: generates all distinct ways of assigning derivatives to fields in field_multiset. The fields
    # S, V, T, Vp, Sp all contain two fields, a psi and a psi_bar. A derivative may be placed on either psi or psi_bar. 
    # More specifically, generate all 'outer partitions' among the field types - in the above example, F, psi_bar_S, psi_S, psi_bar_T, psi_T.
    # For each outer partition, generate all 'inner_partitions' or 'subpartitions.' Given the 
    # outer partition of n, (n1, n2, n3, n4, n5), where each of the five fields fi occurs with multiplicity mi, a 
    # subpartition is a list of tuples [(), (), (), (), ()], where the ith tuple is a sorted partition of ni with
    # mi elements. The function generates all such lists of tuples
    
    #print(field_multiset)
    # extract number of derivatives
    n_D = field_multiset['D']
    
    # find length of outer partition 
    bilinear_keys = [k for k in list(field_multiset.keys()) if k != 'D' and k != 'F'] # deri
    bilinear_multiplicities = [field_multiset[m] for m in list(field_multiset.keys()) if m != 'D' and m != 'F']
    #n_fields = sum([1 if 'F' in field_multiset.keys() else 0]) \ # use list comprehension on list of one element 
    #    + sum([2 for key in bilinears]) # group psi and psi_bar together as one field for now
    
    if 'F' in field_multiset.keys():
        F_multiplicity = [field_multiset['F']]
    else: 
        F_multiplicity = []
    #bilinear_multiplicities = [x for pair in zip(bilinear_multiplicities,bilinear_multiplicities) for x in pair] 
    # for each bilinear, multiplicities are the same for psi and psi_bar
    field_multiplicities = [field_multiset[x] for x in field_multiset.keys() if x != 'D']
    
    n_fields = len(field_multiplicities)
    #print('n_fields: ' + str(n_fields))
    #print('bilinear_multiplicities: ' + str(bilinear_multiplicities))
    #print('field_multiplicities: ' + str(field_multiplicities))
    
    
    #list of lists, each sublist an outer partition, where different orderings are distinct
    outer_partitions_list = list(tuples_sum(n_fields, n_D, order=False)) 
    
    # for each outer partition, generate a list of sorted inner partitions (ordering does not matter, so one may 
    # as well sort them) where the sum of each inner partition is the corresponding element of the outer partition.
    # this is a list of lists of lists.
    #inner_partitions_dict = {}
    derivative_assignments_dict = {}
    for i in range(len(outer_partitions_list)):
        outer_partition = outer_partitions_list[i]
        #print('outer_partition: ' + str(outer_partition))
        inner_partitions_list = generate_inner_partitions(outer_partition, field_multiplicities)
        #inner_partitions_dict[outer_partition] = inner_partitions_list
        # for each outer partition and inner partition, generate a list of pair partitions
        inner_partitions_dict = {}
        for j in range(len(inner_partitions_list)):
            inner_partition = tuple(inner_partitions_list[j])
            #print('inner_partition: ' + str(inner_partition))
            pair_partitions_list = generate_extended_pair_partitions(inner_partition, 'F' in field_multiset.keys())
            inner_partitions_dict[inner_partition] = pair_partitions_list
        derivative_assignments_dict[outer_partition] = inner_partitions_dict

    return derivative_assignments_dict

Note that in cases where there is an $F$ field present, listed pair partitions below correspond only to elements of the corresponding inner partition after the first, since $F$ fields cannot be split into $\bar{\psi}$ and $\psi$ components. 

In [15]:
field_multiset = {'D':2, 'F':1, 'S':1, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (2, 0, 0)
inner_partition: ((2,), (0,), (0, 0))
[[(0, 0)], [(0, 0), (0, 0)]]


outer_partition: (0, 2, 0)
inner_partition: ((0,), (2,), (0, 0))
[[(2, 0)], [(0, 0), (0, 0)]]
[[(0, 2)], [(0, 0), (0, 0)]]
[[(1, 1)], [(0, 0), (0, 0)]]


outer_partition: (0, 0, 2)
inner_partition: ((0,), (0,), (2, 0))
[[(0, 0)], [(2, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (0, 0)]]
[[(0, 0)], [(1, 1), (0, 0)]]
inner_partition: ((0,), (0,), (1, 1))
[[(0, 0)], [(1, 0), (1, 0)]]
[[(0, 0)], [(1, 0), (0, 1)]]
[[(0, 0)], [(0, 1), (0, 1)]]


outer_partition: (1, 1, 0)
inner_partition: ((1,), (1,), (0, 0))
[[(1, 0)], [(0, 0), (0, 0)]]
[[(0, 1)], [(0, 0), (0, 0)]]


outer_partition: (1, 0, 1)
inner_partition: ((1,), (0,), (1, 0))
[[(0, 0)], [(1, 0), (0, 0)]]
[[(0, 0)], [(0, 1), (0, 0)]]


outer_partition: (0, 1, 1)
inner_partition: ((0,), (1,), (1, 0))
[[(1, 0)], [(1, 0), (0, 0)]]
[[(1, 0)], [(0, 1), (0, 0)]]
[[(0, 1)], [(1, 0), (0, 0)]]
[[(0, 1)], [(0, 1), (0, 0)]]


In [16]:
#field_multiset = {'D':4, 'F':1, 'S':1, 'V': 2}
field_multiset = {'D':5, 'F':1, 'S':1, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (5, 0, 0)
inner_partition: ((5,), (0,), (0, 0))
[[(0, 0)], [(0, 0), (0, 0)]]


outer_partition: (0, 5, 0)
inner_partition: ((0,), (5,), (0, 0))
[[(5, 0)], [(0, 0), (0, 0)]]
[[(0, 5)], [(0, 0), (0, 0)]]
[[(4, 1)], [(0, 0), (0, 0)]]
[[(1, 4)], [(0, 0), (0, 0)]]
[[(3, 2)], [(0, 0), (0, 0)]]
[[(2, 3)], [(0, 0), (0, 0)]]


outer_partition: (0, 0, 5)
inner_partition: ((0,), (0,), (5, 0))
[[(0, 0)], [(5, 0), (0, 0)]]
[[(0, 0)], [(0, 5), (0, 0)]]
[[(0, 0)], [(4, 1), (0, 0)]]
[[(0, 0)], [(1, 4), (0, 0)]]
[[(0, 0)], [(3, 2), (0, 0)]]
[[(0, 0)], [(2, 3), (0, 0)]]
inner_partition: ((0,), (0,), (4, 1))
[[(0, 0)], [(4, 0), (1, 0)]]
[[(0, 0)], [(4, 0), (0, 1)]]
[[(0, 0)], [(0, 4), (1, 0)]]
[[(0, 0)], [(0, 4), (0, 1)]]
[[(0, 0)], [(3, 1), (1, 0)]]
[[(0, 0)], [(3, 1), (0, 1)]]
[[(0, 0)], [(1, 3), (1, 0)]]
[[(0, 0)], [(1, 3), (0, 1)]]
[[(0, 0)], [(2, 2), (1, 0)]]
[[(0, 0)], [(2, 2), (0, 1)]]
inner_partition: ((0,), (0,), (3, 2))
[[(0, 0)], [(3, 0), (2, 0)]]
[[(0, 0)], [(3, 0), (0, 2)]

In case below, note that there is no $F$ field, so the number of lists in the pair partition matches the number of elements of the corresponding inner partition.

In [17]:
field_multiset = {'D':3, 'S':1, 'V': 2, 'T': 3}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (3, 0, 0)
inner_partition: ((3,), (0, 0), (0, 0, 0))
[[(3, 0)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 3)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(2, 1)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(1, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 3, 0)
inner_partition: ((0,), (3, 0), (0, 0, 0))
[[(0, 0)], [(3, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 3), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 1), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 2), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((0,), (2, 1), (0, 0, 0))
[[(0, 0)], [(2, 0), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 0), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (0, 1)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 0, 3)
inner_partitio

## Reducing the List of Derivative Assignments with IBP Relations

In [38]:
def IBP_remove(derivative_assignment):
    # INPUT: 
    # derivative_assignment: list of lists of tuples, each sublist corresponding to a different inner partition,
    # each 2-tuple a partition of the corresponding element of the inner partition
    # OUTPUT: 
    # remove_bool: True if derivative_assignment meets requirements for removal via IBP, False otherwise
    # EXPLANATION: 
    # checks whether a derivative assignment is to be removed under the specified prescription - i.e.,
    # whether the maximum number of derivatives acting on any field is unique, and whether any field that comes 
    # earlier in the ordering of field types has exactly one fewer derivative acting on it.
    
    # first, check that maximal number of derivatives only acts on one field
    flattened_list = [x for inner_partition in derivative_assignment for pair in inner_partition for x in pair]
    #print(flattened_list)
    max_derivs = max(flattened_list)
    unique_max = sum([x == max_derivs for x in flattened_list]) == 1
    #print('unique_max: ' + str(unique_max ))
    
    # second, check that no field before this field has one fewer derivative acting on it
    max_index = flattened_list.index(max_derivs)
    #print('max_index: ' + str(max_index))
    if max_index > 0:
        one_less_before = max(flattened_list[:max_index]) == max_derivs - 1
    else:
        one_less_before = False
    #print('one_less_before: ' + str(one_less_before))
    
    if unique_max and not one_less_before:
        return True
    
    return False


def IBP_reduce(derivative_assignments_dict):
    # INPUT:
    # derivative_assignments_dict: dictionary containing all possible derivative assignments for a given
    # field/derivative multiset. 
    # OUTPUT:
    # reduced_derivative_assignments_dict: a dictionary containing an IBP-reduced set of derivative assignments
    # removed_derivative_assignments_dict: a dictionary containing all removed derivative assignments
    # EXPLANATION:
    # scans through all derivative assignments and performs IBP reduction. it removes one operator for each 
    # IBP relation, which appears in one and only one IBP relation. in practice, it does this by removing 
    # any derivative assignments for which the maximum number of derivatives acting on any field is unique, and
    # for which there is not one fewer derivative acting on any field that occurs earlier in the specified ordering
    # of field types. it is worth noting that there is an element of conventionality to this prescription, and that 
    # other IBP reductions are possible that remove different sets of derivative assignments. 
    
    #derivative_assignments_dict_IBP_reduced
    deleted = []
    
    for outer_partition in derivative_assignments_dict.keys():
        inner_partitions_dict = derivative_assignments_dict[outer_partition] 
        for inner_partition in inner_partitions_dict.keys():
            extended_pair_partitions_list = inner_partitions_dict[inner_partition]
            for derivative_assignment in extended_pair_partitions_list:
                if IBP_remove(derivative_assignment):
                    print('REMOVE: ' + str(derivative_assignment))
                    deleted.append(derivative_assignment)
                    print(len(derivative_assignments_dict[outer_partition][inner_partition]))
                    extended_pair_partitions_list.remove(derivative_assignment) 
                    print(len(derivative_assignments_dict[outer_partition][inner_partition]))
        
    return derivative_assignments_dict, deleted





In [39]:
#derivative_assignment = [[(0, 0)], [(2, 0), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
#derivative_assignment = [[(1, 2)], [(0, 0), (0, 0)], [(0, 1), (0, 0), (0, 0)]]
#derivative_assignment = [[(3, 2)], [(0, 0), (0, 0)]]
#derivative_assignment = [[(0, 0)], [(1, 4), (0, 0)]]
#derivative_assignment = [[(1, 0)], [(2, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
#derivative_assignment = [[(0, 0)], [(2, 2), (1, 0)]]
#derivative_assignment = [[(0, 0)], [(2, 1), (2, 0)]]
derivative_assignment = [[(0, 0)], [(1, 3), (1, 0)]]
IBP_remove(derivative_assignment)

True

In [41]:
field_multiset = {'D':5, 'F':1, 'S':1, 'V': 2}
derivative_assignments = generate_derivative_assignments(field_multiset)
derivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)
len(deleted)

REMOVE: [[(5, 0)], [(0, 0), (0, 0)]]
6
5
REMOVE: [[(4, 1)], [(0, 0), (0, 0)]]
5
4
REMOVE: [[(3, 2)], [(0, 0), (0, 0)]]
4
3
REMOVE: [[(0, 0)], [(5, 0), (0, 0)]]
6
5
REMOVE: [[(0, 0)], [(4, 1), (0, 0)]]
5
4
REMOVE: [[(0, 0)], [(3, 2), (0, 0)]]
4
3
REMOVE: [[(0, 0)], [(4, 0), (1, 0)]]
10
9
REMOVE: [[(0, 0)], [(0, 4), (1, 0)]]
9
8
REMOVE: [[(0, 0)], [(3, 1), (1, 0)]]
8
7
REMOVE: [[(0, 0)], [(1, 3), (1, 0)]]
7
6
REMOVE: [[(0, 0)], [(3, 0), (2, 0)]]
12
11
REMOVE: [[(0, 0)], [(3, 0), (1, 1)]]
11
10
REMOVE: [[(0, 0)], [(0, 3), (0, 2)]]
10
9
REMOVE: [[(0, 0)], [(2, 1), (1, 1)]]
9
8
REMOVE: [[(1, 0)], [(0, 0), (0, 0)]]
2
1
REMOVE: [[(4, 0)], [(0, 0), (0, 0)]]
5
4
REMOVE: [[(3, 1)], [(0, 0), (0, 0)]]
4
3
REMOVE: [[(0, 0)], [(4, 0), (0, 0)]]
5
4
REMOVE: [[(0, 0)], [(3, 1), (0, 0)]]
4
3
REMOVE: [[(0, 0)], [(3, 0), (1, 0)]]
8
7
REMOVE: [[(0, 0)], [(0, 3), (1, 0)]]
7
6
REMOVE: [[(0, 0)], [(2, 1), (1, 0)]]
6
5
REMOVE: [[(0, 0)], [(2, 0), (1, 1)]]
6
5
REMOVE: [[(0, 0)], [(0, 2), (1, 1)]]
5
4
REMOVE: [[

68

In [42]:
# compare derivative_assignments and derivative_assignments_IBP_reduced
for outer_partition in derivative_assignments.keys():
    for inner_partition in derivative_assignments[outer_partition].keys():
        print(len(derivative_assignments[outer_partition][inner_partition]))
print('')
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        print(len(derivative_assignments_IBP_reduced[outer_partition][inner_partition]))
'''
print('')
num_removed = 0
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        num_removed += len(derivative_assignments[outer_partition][inner_partition]) - len(derivative_assignments_IBP_reduced[outer_partition][inner_partition])
print(num_removed)
'''
        

1
3
3
6
8
1
2
3
3
5
4
6
6
12
12
2
2
3
2
2
4
8
7
11
16
4
5
6
12
4
6
6
7
6

1
3
3
6
8
1
2
3
3
5
4
6
6
12
12
2
2
3
2
2
4
8
7
11
16
4
5
6
12
4
6
6
7
6


"\nprint('')\nnum_removed = 0\nfor outer_partition in derivative_assignments_IBP_reduced.keys():\n    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():\n        num_removed += len(derivative_assignments[outer_partition][inner_partition]) - len(derivative_assignments_IBP_reduced[outer_partition][inner_partition])\nprint(num_removed)\n"

In [51]:
field_multiset = {'D':2, 'F':1, 'S':1, 'V': 1}
derivative_assignments = generate_derivative_assignments(field_multiset)
print(derivative_assignments)


{(2, 0, 0): {((2,), (0,), (0,)): [[[(0, 0)], [(0, 0)]]]}, (0, 2, 0): {((0,), (2,), (0,)): [[[(2, 0)], [(0, 0)]], [[(0, 2)], [(0, 0)]], [[(1, 1)], [(0, 0)]]]}, (0, 0, 2): {((0,), (0,), (2,)): [[[(0, 0)], [(2, 0)]], [[(0, 0)], [(0, 2)]], [[(0, 0)], [(1, 1)]]]}, (1, 1, 0): {((1,), (1,), (0,)): [[[(1, 0)], [(0, 0)]], [[(0, 1)], [(0, 0)]]]}, (1, 0, 1): {((1,), (0,), (1,)): [[[(0, 0)], [(1, 0)]], [[(0, 0)], [(0, 1)]]]}, (0, 1, 1): {((0,), (1,), (1,)): [[[(1, 0)], [(1, 0)]], [[(1, 0)], [(0, 1)]], [[(0, 1)], [(1, 0)]], [[(0, 1)], [(0, 1)]]]}}


In [52]:
# compare derivative_assignments and derivative_assignments_IBP_reduced
for outer_partition in derivative_assignments.keys():
    for inner_partition in derivative_assignments[outer_partition].keys():
        print(len(derivative_assignments[outer_partition][inner_partition]))

1
3
3
2
2
4


In [49]:
derivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)
len(deleted)

REMOVE: [[(2, 0)], [(0, 0)]]
3
2
REMOVE: [[(0, 0)], [(2, 0)]]
3
2
REMOVE: [[(1, 0)], [(0, 0)]]
2
1


3

In [53]:
derivative_assignments_IBP_reduced

{(2, 0, 0): {((2,), (0,), (0,)): [[[(0, 0)], [(0, 0)]]]},
 (0, 2, 0): {((0,), (2,), (0,)): [[[(0, 2)], [(0, 0)]], [[(1, 1)], [(0, 0)]]]},
 (0, 0, 2): {((0,), (0,), (2,)): [[[(0, 0)], [(0, 2)]], [[(0, 0)], [(1, 1)]]]},
 (1, 1, 0): {((1,), (1,), (0,)): [[[(0, 1)], [(0, 0)]]]},
 (1, 0, 1): {((1,), (0,), (1,)): [[[(0, 0)], [(1, 0)]], [[(0, 0)], [(0, 1)]]]},
 (0,
  1,
  1): {((0,), (1,), (1,)): [[[(1, 0)], [(1, 0)]],
   [[(1, 0)], [(0, 1)]],
   [[(0, 1)], [(1, 0)]],
   [[(0, 1)], [(0, 1)]]]}}

In [54]:
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        print(len(derivative_assignments_IBP_reduced[outer_partition][inner_partition]))

1
2
2
1
2
4
